##드라이브에 학습된 모델과 CSV 파일을 넣으세요
(학습된 모델은 공유해드렸습니다. 구글 드라이브에 가시면 있습니다)

(CSV 파일은 각자 가지고 계신 파일을 사용하시면 됩니다.)
(주영오빠 = real_final_0.csv / 다원이 = real_final_1.csv)
##런타임 -> 런타임 유형변경 -> GPU 변경해주세요

이후부터는 쭉 진행해 주세요



In [ ]:
# Hugging Face의 트랜스포머 모델을 설치
!pip install transformers

     |████████████████████████████████| 1.5MB 13.2MB/s 
     |████████████████████████████████| 2.9MB 55.4MB/s 
     |████████████████████████████████| 890kB 47.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=1ccf7defc1f1406522535d4f306c233ad4cea9807409388cc98e6ce095976222
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [ ]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


###자신의 드라이브 경로를 입력하여 학습된 모델을 불러옵니다



In [ ]:
# 분류를 위한 BERT 모델 생성
model = torch.load('/content/drive/MyDrive/긍우중_4.pth')
model.cuda() #model.to(device) #model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

In [ ]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

###자신의 드라이브 경로를 입력하여 CSV 파일을 불러옵니다

In [ ]:
import pandas as pd
text = pd.read_csv('/content/drive/MyDrive/real_final_0.csv', encoding = 'utf-8')
text

,index,text
0,0,ㅋ 인간 극장 이
1,1,성 석문 숙 가게 드나들 경찰 대해 직무 유기 고발장 냈 대구 수성 경찰 경찰 가게...
2,2,좋 건강 정보 고맙
3,3,뺏 정도 라니 전 킬로 였 인간 승리 네요 응 원합 니 당
4,4,오른쪽 남자 뺀 사람 인 알
...,...,...
455744,456995,나라 응원 일본 아베 응원 이대로 쭉 가
455745,456996,내려와라 이만큼 했 됐 무능 알
455746,456997,중국 온 세계 전쟁 같 고통 겪 코로나 자연 발생 바이러스 같
455747,456998,ㅋㅋㅋ 개소리


In [ ]:
text = text.loc[text['text'].notnull(),:]
text=text.reset_index()
text

,level_0,index,text
0,0,0,ㅋ 인간 극장 이
1,1,1,성 석문 숙 가게 드나들 경찰 대해 직무 유기 고발장 냈 대구 수성 경찰 경찰 가게...
2,2,2,좋 건강 정보 고맙
3,3,3,뺏 정도 라니 전 킬로 였 인간 승리 네요 응 원합 니 당
4,4,4,오른쪽 남자 뺀 사람 인 알
...,...,...,...
454893,455744,456995,나라 응원 일본 아베 응원 이대로 쭉 가
454894,455745,456996,내려와라 이만큼 했 됐 무능 알
454895,455746,456997,중국 온 세계 전쟁 같 고통 겪 코로나 자연 발생 바이러스 같
454896,455747,456998,ㅋㅋㅋ 개소리


####range 범위 신경써서 변경해주세요!!!!!

In [ ]:
result = pd.DataFrame(columns=['result'])
for i in range(0,454898):
  logits = test_sentences([text['text'][i]])
  result = result.append(pd.DataFrame([np.argmax(logits)], columns=['result']), ignore_index=True)


In [ ]:
result

,result
0,2
1,2
2,1
3,1
4,2
...,...
454893,1
454894,2
454895,0
454896,1


In [ ]:
result.reset_index().groupby('result').count()

,index
result,
0,42644
1,46126
2,366128


####불러온 파일의 숫자에 맞게 숫자를 변경하여 저장하세요
ex) real_final_3 이라면 tag_우울_result_3 으로 저장해주세요

In [ ]:
result.to_csv('/content/drive/MyDrive/tag_우울_result_0.csv', encoding = 'utf-8-sig')